*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


In [ ]:
train_data_frame = pd.read_csv(train_file_path, sep='\t',names=["class", "text"], error_bad_lines=False)
test_data_frame = pd.read_csv(test_file_path, sep='\t',names=["class", "text"], error_bad_lines=False)

train_data_frame_copy = train_data_frame.copy()
test_data_frame_copy = test_data_frame.copy()

print(train_data_frame_copy.tail())
print(test_data_frame_copy.tail())

from sklearn.model_selection import train_test_split

train_data_frame_copy, val_data_frame_copy = train_test_split(train_data_frame_copy, test_size=0.2)

train_data_frame_copy['class'] = pd.Categorical(train_data_frame_copy['class'])
train_data_frame_copy['class'] = train_data_frame_copy['class'].cat.codes

val_data_frame_copy['class'] = pd.Categorical(val_data_frame_copy['class'])
val_data_frame_copy['class'] = val_data_frame_copy['class'].cat.codes

test_data_frame_copy['class'] = pd.Categorical(test_data_frame_copy['class'])
test_data_frame_copy['class'] = test_data_frame_copy['class'].cat.codes

# print(train_data_frame_copy.columns)
# train_target = train_data_frame_copy.pop('class')
# print(train_data_frame_copy.tail(), train_data_frame_copy.shape)
# print('----')
# print(train_target.tail(), train_target.shape)
# print('----')
# test_target = test_data_frame_copy.pop('class')
# print(test_data_frame_copy.tail(), test_data_frame_copy.shape)
# print('----')
# print(test_target.tail(), test_target.shape)
# print('----')


# train_target.plot(kind='hist', bins=2, title='Classes in train dataset')
# test_target.plot(kind='hist', bins=2, title='Classes in test dataset')

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAXLEN = 50
BATCH_SIZE = 32 

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data_frame_copy['text'])

X_train = tokenizer.texts_to_sequences(train_data_frame_copy['text'])
X_val = tokenizer.texts_to_sequences(val_data_frame_copy['text'])
X_test = tokenizer.texts_to_sequences(test_data_frame_copy['text'])

VOCAB_SIZE = len(tokenizer.word_index) + 1

print(train_data_frame_copy['text'])
print(X_train[2])

X_train = pad_sequences(X_train, padding='post', maxlen=MAXLEN)
X_val = pad_sequences(X_val, padding='post', maxlen=MAXLEN)
X_test = pad_sequences(X_test, padding='post', maxlen=MAXLEN)

y_train = train_data_frame_copy['class']
y_val = val_data_frame_copy['class']
y_test = test_data_frame_copy['class']

print(X_train[0, :])

In [ ]:
from keras.models import Sequential
from keras import layers

EMBEDDING_DIM = 100

model = Sequential()
model.add(layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAXLEN))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.build((None, MAXLEN))
model.summary()


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_val, y_val),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))



In [ ]:
plot_history(history)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  text = tokenizer.texts_to_sequences([pred_text])
  text = pad_sequences(text, padding='post', maxlen=MAXLEN)
  prediction = model.predict(text)

  if prediction<0.5:
     return (prediction[0, 0], 'ham')
  else:
    return  (prediction[0, 0], 'spam')

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(prediction)
      print(msg)
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
